<a href="https://colab.research.google.com/github/carrie86/Project_maskrcnn/blob/main/demo_apple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/Pro_apple/Mask_RCNN/samples

/content/drive/My Drive/Pro_apple/Mask_RCNN/samples


In [4]:
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 39kB/s 
     |████████████████████████████████| 512kB 44.1MB/s 
     |████████████████████████████████| 3.8MB 44.7MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=7d381493fe2b9678d10f676463586c91962a9a5d3774e7530601145645d79c6d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [5]:
!pip install keras==2.3.1

     |████████████████████████████████| 378kB 3.4MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [13]:
!pip install wandb

     |████████████████████████████████| 1.7MB 3.2MB/s 
     |████████████████████████████████| 133kB 15.7MB/s 
     |████████████████████████████████| 102kB 7.4MB/s 
     |████████████████████████████████| 163kB 15.1MB/s 
     |████████████████████████████████| 102kB 7.0MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=06f1e94fbd660011df35be3c730f265cb7188d5d579ce2db92cc380db7787508
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=5bb008faeef97e83ac8eed597d6d119200b9fb5912ee04269b32e62b37f0700d
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=b2f013f7d06695d4a34a6fa44fc08fe88ba85c958f14bc

In [14]:
%cd /content/drive/My Drive/Pro_apple/Mask_RCNN/samples/apple

/content/drive/My Drive/Pro_apple/Mask_RCNN/samples/apple


In [15]:
!wandb login d72b44fadcf6d4ea71e979875a5169e4c3efd698

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [16]:
import wandb
!wandb.init(project="apple_diseases_detection",name=f"{datetime.datetime.now()}")

/bin/bash: -c: line 0: syntax error near unexpected token `project="apple_diseases_detection",name=f"{datetime.datetime.now()}"'
/bin/bash: -c: line 0: `wandb.init(project="apple_diseases_detection",name=f"{datetime.datetime.now()}")'


In [30]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
# sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
# import coco
import apple.apple_wandb as apple
from tqdm import tqdm #<추가

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR,"logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "logs/apple20201104T0127/mask_rcnn_apple_0061.h5")
print(COCO_MODEL_PATH)
# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
#   utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

/content/drive/My Drive/Pro_apple/Mask_RCNN/samples/logs/apple20201104T0127/mask_rcnn_apple_0061.h5


## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [31]:
class InferenceConfig(apple.AppleConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.5
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             3.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

## Create Model and Load Trained Weights

In [32]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

OSError: ignored

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the MS-COCO dataset, the 'person' class is 1 and 'teddy bear' is 88. The IDs are often sequential, but not always. The COCO dataset, for example, has classes associated with class IDs 70 and 72, but not 71.

To improve consistency, and to support training on data from multiple sources at the same time, our ```Dataset``` class assigns it's own sequential integer IDs to each class. For example, if you load the COCO dataset using our ```Dataset``` class, the 'person' class would get class ID = 1 (just like COCO) and the 'teddy bear' class is 78 (different from COCO). Keep that in mind when mapping class IDs to class names.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load COCO dataset
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

We don't want to require you to download the COCO dataset just to run this demo, so we're including the list of class names below. The index of the class name in the list represent its ID (first class is 0, second is 1, third is 2, ...etc.)

In [ ]:
class_names =['Sooty blotch', 'Bitter rot','brown rot']

## Run Object Detection

In [ ]:
# Load a random image from the images folder

filename = os.path.join(IMAGE_DIR, 'brown_rot_apple1.jpg')
image= skimage.io.imread(filename)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)

In [ ]:
# Run detection
results = model.detect([image], verbose=1)

# Visualize results

print(results)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

In [ ]:
 r['rois']

In [23]:
r['masks']

NameError: ignored

In [24]:
r['class_ids']

NameError: ignored

In [25]:
print(r['scores'])

NameError: ignored